### Imports

In [ ]:
import json
import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets

from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap

from qiskit.opflow import X, Y, Z, I, CircuitStateFn
from qiskit.opflow.state_fns import StateFn
from qiskit.opflow.expectations import PauliExpectation, MatrixExpectation
from qiskit.opflow.converters import CircuitSampler
from qiskit.providers.aer import QasmSimulator

from sklearn import datasets
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn import metrics

import matplotlib.cm as cm

%matplotlib inline

### Functions

In [ ]:
def even(x):
    return (x % 2) == 0


def generate_pauli(n):
    hamiltonians = []

    h = 1
    for _ in range(n):
        h = h^Z
    hamiltonians.append(h)

    for k in range(n):
       
        h = 1

        if(k != 0):
            for _ in range(k):
                h = h^X

        if even(k+1):
            h = h^X
        else:
            h = h^Y

        if (k+1 != n):
            for _ in range(k+1, n):
                h = h^Z


        hamiltonians.append(h)

    return hamiltonians


def purity_score(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

In [ ]:
def ag_clustering(dataset, classes, dissimilarity_matrix, linkage="complete", title=""):
    clustering = AgglomerativeClustering(affinity='precomputed', n_clusters=np.unique(classes).size,  linkage=linkage).fit(dissimilarity_matrix)

    labels = clustering.labels_
    cmap = plt.cm.get_cmap("Accent").copy()
    cmap.set_under('red')
    contingency_matrix = metrics.cluster.contingency_matrix(classes, labels)

    fig, axs = plt.subplots(1,2)
    fig.suptitle(title)
    fig.set_size_inches(15,6)

    axs[0].set_title("Contingency Matrix")
    axs[0].matshow(contingency_matrix, cmap='RdPu')
    axs[1].set_title("Predicted Clusters")
    axs[1].scatter(dataset[:,0], dataset[:,1], c=labels, cmap=cmap)


def dbscan_clustering(dataset, classes, dissimilarity_matrix, eps=0.5, min_samples=5, title=""):
    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed').fit(dissimilarity_matrix)

    labels = clustering.labels_
    cmap = plt.cm.get_cmap("Accent").copy()
    cmap.set_under('red')
    contingency_matrix = metrics.cluster.contingency_matrix(classes, labels)

    fig, axs = plt.subplots(1,2)
    fig.suptitle(title)
    fig.set_size_inches(15,6)

    axs[0].set_title("Contingency Matrix")
    axs[0].matshow(contingency_matrix, cmap='RdPu')
    axs[1].set_title("Predicted Clusters")
    axs[1].scatter(dataset[:,0], dataset[:,1], c=labels, cmap=cmap)

In [ ]:
def em_distance_approximate(data_1, data_2, encode_map, observables, simulator):

    x = CircuitStateFn(encode_map.bind_parameters(data_1))
    y = CircuitStateFn(encode_map.bind_parameters(data_2))

    em = 0

    for h in observables:

        measurable_x = StateFn(h, is_measurement=True).compose(x)
        measurable_y = StateFn(h, is_measurement=True).compose(y)

        expectation_x = PauliExpectation().convert(measurable_x)
        expectation_y = PauliExpectation().convert(measurable_y)

        sampler_x = CircuitSampler(simulator).convert(expectation_x)
        sampler_y = CircuitSampler(simulator).convert(expectation_y)

        current_expectation = abs(sampler_x.eval().real - sampler_y.eval().real)

        if current_expectation > em:
            em = current_expectation

    return em

def em_distance_exact(data_1, data_2, encode_map, observables):

    x = CircuitStateFn(encode_map.bind_parameters(data_1))
    y = CircuitStateFn(encode_map.bind_parameters(data_2))

    em = 0

    for h in observables:

        circuit_x = x.adjoint().compose(h).compose(x)
        circuit_y = y.adjoint().compose(h).compose(y)

        current_expectation = abs(circuit_x.eval().real - circuit_y.eval().real)

        if current_expectation > em:
            em = current_expectation

    return em


def em_dissimilarity_matrix(dataset, approximate = False):

    encode_map = ZZFeatureMap(feature_dimension=dataset.shape[1], reps=1, entanglement='linear', insert_barriers=True)
    n = encode_map.num_qubits
    observables = generate_pauli(n)
    mat = np.zeros((dataset.shape[0], dataset.shape[0]))

    if approximate:
        simulator = QasmSimulator()

        for i in range(dataset.shape[0]):
            for j in range(i):
                dis = em_distance_approximate(dataset[i], dataset[j], encode_map, observables, simulator)
                mat[i,j] = dis
                mat[j,i] = dis
    else:

        for i in range(dataset.shape[0]):
            for j in range(i):
                dis = em_distance_exact(dataset[i], dataset[j], encode_map, observables)
                mat[i,j] = dis
                mat[j,i] = dis

    return mat

def euclidean_matrix(dataset):
    mat = np.zeros((dataset.shape[0], dataset.shape[0]))

    for i in range(dataset.shape[0]):
        for j in range(i):
            dis = np.sqrt(np.sum((dataset[i] - dataset[j])**2))
            mat[i,j] = dis
            mat[j,i] = dis

    return mat


def manhattan_matrix(dataset):
    mat = np.zeros((dataset.shape[0], dataset.shape[0]))

    for i in range(dataset.shape[0]):
        for j in range(i):
            dis = np.sum(np.abs(dataset[i] - dataset[j]))
            mat[i,j] = dis
            mat[j,i] = dis

    return mat

# Clustering

## Blob dataset

In [ ]:
TRAIN_SIZE = 20

dataset, classes = datasets.make_blobs(n_samples=TRAIN_SIZE, random_state=10, centers=2, n_features=2)
dataset = np.array(dataset)

plt.scatter(dataset[:,0], dataset[:,1], c=classes, cmap="Accent")

In [ ]:
em_diss = em_dissimilarity_matrix(dataset, approximate=False)
l2_diss = euclidean_matrix(dataset)
l1_diss = manhattan_matrix(dataset)

In [ ]:


fig, axs = plt.subplots(1,3)
fig.set_size_inches(15,6)

axs[0].matshow(em_diss, cmap='RdPu')
axs[0].set_title("EM Distance")
axs[1].matshow(l1_diss, cmap='RdPu')
axs[1].set_title("Manhattan Distance")
axs[2].matshow(l2_diss, cmap='RdPu')
axs[2].set_title("Euclidean Distance")

fig.show()

### Agglomerative clustering

In [ ]:
ag_clustering(dataset = dataset, classes = classes, dissimilarity_matrix = em_diss, linkage="complete", title = "EM")
ag_clustering(dataset = dataset, classes = classes, dissimilarity_matrix = l2_diss, linkage="complete", title = "Euclidean")

### DBSCAN

In [ ]:
dbscan_clustering(dataset=dataset, classes=classes, dissimilarity_matrix=em_diss, eps=2, min_samples=10, title="EM")
dbscan_clustering(dataset=dataset, classes=classes, dissimilarity_matrix=l2_diss, eps=5, min_samples=20, title="Euclidean")
